In [16]:
import os
import pandas as pd
from tqdm import tqdm
from llama_cpp import Llama
from llama_cpp.llama_chat_format import Llava15ChatHandler
import pathlib
from PIL import Image
import io

# --- 1. Конфигурация ---
MODEL_PATH = "./models/llava-v1.6-mistral-7b.Q4_K_M.gguf" 
CLIP_MODEL_PATH = "./models/mmproj-model-f16.gguf"
IMG_DIR = "img"
TEMP_IMG_DIR = "img_resized" 
os.makedirs(TEMP_IMG_DIR, exist_ok=True)
RESULTS_FILE = "llava_local_7b_detailed_results.csv" # Новое имя файла

# <-- 1. НОВЫЙ, СТРУКТУРИРОВАННЫЙ "CHAIN OF THOUGHT" ПРОМПТ
# Мы даем модели шаблон, который она должна заполнить.
PROMPT = """Ты — эксперт по определению пород кошек. Проанализируй изображение по следующему плану:

1.  **Анализ признаков:** Опиши одним-двумя словами ключевые видимые признаки кошки (форма головы, уши, шерсть, телосложение, цвет глаз).
2.  **Сравнение с породой "Британская короткошерстная":** Сравни проанализированные признаки с эталоном британской породы.
3.  **Итоговый вердикт:** Дай короткий ответ "Да" или "Нет".
4.  **Альтернативная порода (если вердикт "Нет"):** Если это не британская кошка, укажи наиболее вероятную породу.

Ответ дай строго по этому плану."""

# --- 2. Настройки производительности и генерации ---
N_GPU_LAYERS = -1
N_CTX = 4096
N_BATCH = 1024
N_THREADS = os.cpu_count()

# <-- 2. ИЗМЕНЯЕМ ПАРАМЕТРЫ ГЕНЕРАЦИИ ДЛЯ РАЗВЕРНУТОГО ОТВЕТА
TEMPERATURE = 0.3     # Чуть больше "творчества" для описаний
REPEAT_PENALTY = 1.1
MAX_TOKENS = 300      # <-- Увеличиваем лимит, чтобы поместился развернутый ответ

# --- 3. Функция сжатия (без изменений) ---
def resize_and_save_image(original_path, output_path, max_size=(1024, 1024)):
    with Image.open(original_path) as img:
        img.thumbnail(max_size)
        if img.mode in ("RGBA", "P"): img = img.convert("RGB")
        img.save(output_path, format='JPEG')
    return output_path

# --- 4. Загрузка модели (без изменений) ---
llm = None
if os.path.exists(MODEL_PATH) and os.path.exists(CLIP_MODEL_PATH):
    print("Загрузка моделей...")
    try:
        chat_handler = Llava15ChatHandler(clip_model_path=CLIP_MODEL_PATH, verbose=False)
        llm = Llama(
            model_path=MODEL_PATH, chat_handler=chat_handler,
            n_ctx=N_CTX, n_gpu_layers=N_GPU_LAYERS, n_batch=N_BATCH, n_threads=N_THREADS, verbose=False
        )
        print("✅ Модели успешно загружены!")
    except Exception as e:
        print(f"❌ Ошибка загрузки моделей: {e}")
else:
    print(f"❌ Файл основной модели или CLIP не найден.")

# --- 5. Основной цикл обработки (без изменений, кроме вызова llm) ---
if llm:
    try:
        image_files = sorted([f for f in os.listdir(IMG_DIR) if f.endswith(('.jpg', '.jpeg', '.png'))])
        results_data = []
        
        print(f"Начинаем обработку {len(image_files)} изображений для получения развернутых ответов...")
        for filename in tqdm(image_files, desc="LLaVA Local 7B (Detailed)"):
            original_image_path = os.path.join(IMG_DIR, filename)
            temp_image_path = os.path.join(TEMP_IMG_DIR, filename)
            try:
                resize_and_save_image(original_image_path, temp_image_path)
                image_uri = pathlib.Path(temp_image_path).resolve().as_uri()
                
                prompt_for_llm = f"USER: <image>\n{PROMPT}\nASSISTANT:"
                
                response = llm(
                    prompt_for_llm,
                    stop=["USER:"], # Достаточно одного стоп-слова
                    max_tokens=MAX_TOKENS,
                    temperature=TEMPERATURE,
                    repeat_penalty=REPEAT_PENALTY
                )
                
                result_text = response['choices'][0]['text'].strip()
                results_data.append({"filename": filename, "llava_7b_response": result_text})

            except Exception as e:
                results_data.append({"filename": filename, "llava_7b_response": f"Error: {e}"})
        
        df_results = pd.DataFrame(results_data)
        df_results.to_csv(RESULTS_FILE, index=False, encoding='utf-8')
        print(f"\n✅ Результаты сохранены в '{RESULTS_FILE}'.")
        display(df_results.head())

    except FileNotFoundError:
        print(f"❌ Ошибка: Директория '{IMG_DIR}' не найдена.")
else:
    print("\n❌ Основной процесс не был запущен, так как модели не были загружены.")

Загрузка моделей...


llama_new_context_with_model: n_ctx_per_seq (4096) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


✅ Модели успешно загружены!
Начинаем обработку 30 изображений для получения развернутых ответов...


LLaVA Local 7B (Detailed): 100%|██████████| 30/30 [29:59<00:00, 59.98s/it]


✅ Результаты сохранены в 'llava_local_7b_detailed_results.csv'.


,filename,llava_7b_response
0,brit1.jpg,1. **Анализ признаков:** Ключевые видимые при...
1,brit10.jpg,1. **Анализ признаков:** Ключевые видимые приз...
2,brit2.jpg,1. **Анализ признаков:** Кошка в изображении ...
3,brit3.jpg,1. **Анализ признаков:** В изображении видны к...
4,brit4.jpg,1. В изображении видны ключевые признаки: кошк...
